In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl



In [2]:
def formalize_data (bolillas_str: str) -> tuple[str, tuple[int]]:
  arr = bolillas_str.split(" ")
  arr.sort()
  combo = tuple(int(a) for a in arr)
  id = "".join(arr)
  return id, combo

a = formalize_data("09 12 27 33 24 30")
print(a)

('091224273033', (9, 12, 24, 27, 30, 33))


In [3]:
from enum import Enum

class RARE_REASONS(Enum):
  MAX_PRIMES = 1
  HAVENT_EVENS = 2
  HAVENT_ODDS = 4

# Lista de números primos entre 1 y 50
PRIME_NUMBERS = {2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47}

# Configuración
TOTAL_NUMBERS = 50
CHOOSE = 6

# Rangos de valores de las bolillas b1 a b6
MIN_B_VALUES = [1, 2, 3, 4, 5, 6]
MAX_B_VALUES = [45, 46, 47, 48, 49, 50]

# Función para determinar si un número es primo
# Utilizando una tabla precalculada para mayor eficiencia
def is_prime(num):
  return num in PRIME_NUMBERS

# Función para determinar si una combinación es "rara"
def is_rare_combination(combo) -> tuple[bool, int]:
  reasons = 0

  # Regla 1: Máximo 3 números primos
  prime_count = sum(1 for num in combo if is_prime(num))
  if prime_count > 3:
    reasons += RARE_REASONS.MAX_PRIMES.value

  # Regla 2: Al menos un número par
  if not any(num % 2 == 0 for num in combo):
    reasons += RARE_REASONS.HAVENT_EVENS.value

  # Regla 3: Al menos un número impar
  if not any(num % 2 == 1 for num in combo):
    reasons += RARE_REASONS.HAVENT_ODDS.value

  return reasons > 0, reasons


# Calcular probabilidades de izquierda a derecha
def calculate_left_scores(combo):
  left_scores = []

  for i, value in enumerate(combo):
    if i == 0:
      possible_values = combo[i + 1] - 1
    else:
      possible_values = MAX_B_VALUES[i] - (combo[i - 1] + 1) + 1
    
    left_scores.append(1 / possible_values)

  return left_scores

# Calcular probabilidades de derecha a izquierda
def calculate_right_scores(combo):
  right_scores = []

  for i, value in enumerate(combo):
    if (i == CHOOSE - 1):
      possible_values = MAX_B_VALUES[i] - (combo[i - 1] + 1) + 1
    else:
      possible_values = (combo[i + 1] - 1) - MIN_B_VALUES[i] + 1
    
    right_scores.append(1 / possible_values)

  return right_scores


# Calcular los saltos entre valores
def jumps_map (combo: tuple[int]):
  differences = [combo[i + 1] - combo[i] for i in range(CHOOSE-1)]
  # differences.sort()
  result = "_".join([str.zfill(str(d), 2) for d in differences])
  return result


In [4]:
l = [0.0588, 0.0303, 0.0345, 0.0455, 0.0526, 0.0588]
r = [0.0588, 0.0417, 0.0370, 0.0345, 0.0333, 0.0588]

def check_shared_values (left_arr: List[float], right_arr: List[float]):
  shared_values = []
  if (left_arr[0] == right_arr[CHOOSE-1]):
    shared_values.append(left_arr[0])
  if (left_arr[CHOOSE-1] == right_arr[0]):
    shared_values.append(left_arr[CHOOSE-1])

  for i in range(len(left_arr)):
    for j in range(1, len(right_arr) - 1):
      if left_arr[i] == right_arr[j]:
        shared_values.append(left_arr[i])
  return shared_values

# iniciar comparacion desde aqui

In [5]:
# url = 'https://resultados.intralot.com.pe/i.do?m=historico&t=0&s=41'
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=41'
# response = requests.get(url, verify=False)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

historico = soup.find(id = 'historico')
rows = historico.find_all('tr')

In [6]:
lst = []
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int = formalize_data(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  
  is_rare, reason = is_rare_combination(bolillas_int)
  cols.append(is_rare)
  cols.append(reason)

  left_scores = [round(b, 4) for b in calculate_left_scores(bolillas_int)]
  right_scores = [round(b, 4) for b in calculate_right_scores(bolillas_int)]
  lr_scores = [round(left + right,4) for left, right in zip(left_scores, right_scores)]
  row_score = round(sum(lr_scores),4)

  shared_values = list(set(left_scores[0:5]) & set(right_scores[1:6]))
  shared_values = [round(s, 4) for s in shared_values]
  cols.extend([score for score in left_scores])
  cols.extend([score for score in right_scores])
  # cols.extend(lr_scores)
  cols.append(shared_values)
  cols.append(len(shared_values))
  cols.append(row_score)
  cols.append(jumps_map(bolillas_int))
  
  lst.append(cols)

with open('./tinkache.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas', 'yapa', 'adicionales', 'sorteo_extra',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'is_rare_combination', 'reason',
    "score_left_b1", "score_left_b2", "score_left_b3", "score_left_b4", "score_left_b5", "score_left_b6",
    "score_r8_b1", "score_r8_b2", "score_r8_b3", "score_r8_b4", "score_r8_b5", "score_r8_b6",
    # "score_lr_b1", "score_lr_b2", "score_lr_b3", "score_lr_b4", "score_lr_b5", "score_lr_b6",
    "shared_values", "shared_values_count",
    "score_row",
    "jumps_map"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'shared_values_count': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

display(dfRows.head(10))


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_left_b6,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,score_r8_b6,shared_values,shared_values_count,score_row,jumps_map
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,bool,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,str
"""09/02/2025""","""1167""","""34 03 46 35 33 12""","""18""","""44 06 41""","""Promoción Sí o Sí""","""031233343546""","[3, 12, … 46]",3,12,33,34,35,46,false,0,0.0909,0.0233,0.0286,0.0667,0.0667,0.0667,0.0909,0.0323,0.0323,0.0323,0.0244,0.0667,[0.0667],1,0.6218,"""09_21_01_01_11"""
"""05/02/2025""","""1166""","""19 12 27 33 24 30""","""17""","""07""","""Promoción Sí o Sí""","""121924273033""","[12, 19, … 33]",12,19,24,27,30,33,false,0,0.0556,0.0294,0.0357,0.0417,0.0455,0.05,0.0556,0.0455,0.0417,0.0385,0.0357,0.05,"[0.0455, 0.0417, 0.0357]",3,0.5249,"""07_05_03_03_03"""
"""02/02/2025""","""1165""","""20 48 14 12 28 04""","""02""","""40""","""Promoción Sí o Sí""","""041214202848""","[4, 12, … 48]",4,12,14,20,28,48,true,4,0.0909,0.0238,0.0286,0.0294,0.0345,0.0455,0.0909,0.0833,0.0588,0.0417,0.0233,0.0455,[],0,0.5962,"""08_02_06_08_20"""
"""29/01/2025""","""1164""","""33 23 17 13 24 12""","""26""","""38 39 28 29""","""Promoción Sí o Sí""","""121317232433""","[12, 13, … 33]",12,13,17,23,24,33,false,0,0.0833,0.0294,0.0294,0.0323,0.0385,0.0385,0.0833,0.0667,0.05,0.05,0.0357,0.0385,[0.0385],1,0.5756,"""01_04_06_01_09"""
"""26/01/2025""","""1163""","""33 26 35 30 13 18""","""05""","""27 10""","""Promoción Sí o Sí""","""131826303335""","[13, 18, … 35]",13,18,26,30,33,35,false,0,0.0588,0.0303,0.0345,0.0455,0.0526,0.0588,0.0588,0.0417,0.037,0.0345,0.0333,0.0588,"[0.0345, 0.0588]",2,0.5446,"""05_08_04_03_02"""
"""22/01/2025""","""1162""","""21 10 49 31 32 06""","""14""","""39 29 36""","""Promoción Sí o Sí""","""061021313249""","[6, 10, … 49]",6,10,21,31,32,49,false,0,0.1111,0.025,0.027,0.037,0.0556,0.0556,0.1111,0.0526,0.0357,0.0357,0.0227,0.0556,[0.0556],1,0.6247,"""04_11_10_01_17"""
"""19/01/2025""","""1161""","""23 44 45 05 02 01""","""16""","""24 19""","""Promoción Sí o Sí""","""010205234445""","[1, 2, … 45]",1,2,5,23,44,45,false,0,1.0,0.0222,0.0222,0.0233,0.0385,0.1667,1.0,0.3333,0.05,0.025,0.025,0.1667,[],0,2.8729,"""01_03_18_21_01"""
"""15/01/2025""","""1160""","""36 16 47 38 19 46""","""32""","""18 24 43 08""","""Promoción Sí o Sí""","""161936384647""","[16, 19, … 47]",16,19,36,38,46,47,false,0,0.0556,0.0333,0.0357,0.0833,0.0909,0.25,0.0556,0.0294,0.0286,0.0238,0.0238,0.25,[],0,0.96,"""03_17_02_08_01"""
"""12/01/2025""","""1159""","""49 32 41 44 26 34""","""14""","""43""","""Promoción Sí o Sí""","""263234414449""","[26, 32, … 49]",26,32,34,41,44,49,false,0,0.0323,0.05,0.0667,0.0714,0.125,0.1667,0.0323,0.0312,0.0263,0.025,0.0227,0.1667,[],0,0.8163,"""06_02_07_03_05"""


# CHECK IF YOU COULD WON

In [ ]:
def generate_id (int_ticket: list[int]):
  return "".join([str.zfill(str(t), 2)  for t in int_ticket])

def generate_yapa_tickets (int_ticket: list[int], yapa: int, size: int):
  combos_size = list(combinations(int_ticket, size))
  result = []
  
  for i in range(len(combos_size)):
    row = [*combos_size[i], yapa]
    row.sort()
    result.append(row)

  return result

def generate_yapas (int_ticket: list[int], yapa: int):
  yapa_5 = generate_yapa_tickets(int_ticket=int_ticket, yapa=yapa, size=5)
  yapa_4 = generate_yapa_tickets(int_ticket=int_ticket, yapa=yapa, size=4)
  yapa_3 = generate_yapa_tickets(int_ticket=int_ticket, yapa=yapa, size=3)
  yapa_2 = generate_yapa_tickets(int_ticket=int_ticket, yapa=yapa, size=2)

  return yapa_5, yapa_4, yapa_3, yapa_2

def generate_adicionales (int_ticket: list[int], adicionales: list[int]):
  combos_5 = list(combinations(int_ticket, 5))
  result = []
  for i in range(len(combos_5)):
    for j in range(len(adicionales)):
      row = [*combos_5[i], adicionales[j]]
      row.sort()
      result.append(row)

  return result

def is_winner_ticket (df: pl.DataFrame, tickets_id: list[str]):
  a = df.filter(
    pl.col("id").is_in(tickets_id)
  )

  is_winner = len(a) > 0
  if (is_winner):
    print(a)

  return len(a) > 0

def get_tickets_ids (ticket: str, yapa: str, adicionales: str):
  int_ticket = [int(t) for t in ticket.split(" ")]
  int_yapa = int(yapa)
  int_adicionales = [int(a) for a in adicionales.split(" ")]

  int_ticket.sort()

  id_ticket = generate_id(int_ticket)
  yapas_5, yapas_4, yapas_3, yapas_2 = generate_yapas(int_ticket=int_ticket, yapa=int_yapa)
  adicionales = generate_adicionales(int_ticket=int_ticket, adicionales=int_adicionales)

  return id_ticket, \
    [generate_id(y) for y in yapas_5], \
    [generate_id(a) for a in adicionales], \
    [generate_id(y) for y in yapas_4], \
    [generate_id(y) for y in yapas_3], \
    [generate_id(y) for y in yapas_2], \


def check_my_ticket (df: pl.DataFrame, my_ticket=str):
  int_my_ticket = [int(t) for t in my_ticket.split(" ")]
  int_my_ticket.sort()
  id_my_ticket = generate_id(int_my_ticket)

  last_result = df[0]  
  result_bolillas = last_result["bolillas"][0]
  result_yapa = last_result["yapa"][0]
  result_adicionales = last_result["adicionales"][0]

  bolillas_id, yapas_5, adicionales_ids, y4, y3, y2 = get_tickets_ids(ticket=result_bolillas, yapa=result_yapa, adicionales=result_adicionales)

  if (id_my_ticket == bolillas_id):
    print("Holy sht! YOU WON!!!!")
    return

  if (id_my_ticket in yapas_5):
    print("You won with yapa!!!!")
    return

  if (id_my_ticket in adicionales_ids):
    print("You won with adicionales!!!!")
    return
  
  print("Don't worry, you'll do better next time")


my_ticket = "19 12 27 33 24 31"
check_my_ticket(df=dfRows, my_ticket=my_ticket)


['0312183334', '0312183335', '0312183346', '0312183435', '0312183446', '0312183546', '0318333435', '0318333446', '0318333546', '0318343546', '1218333435', '1218333446', '1218333546', '1218343546', '1833343546']
['03121833', '03121834', '03121835', '03121846', '03183334', '03183335', '03183346', '03183435', '03183446', '03183546', '12183334', '12183335', '12183346', '12183435', '12183446', '12183546', '18333435', '18333446', '18333546', '18343546']
['031218', '031833', '031834', '031835', '031846', '121833', '121834', '121835', '121846', '183334', '183335', '183346', '183435', '183446', '183546']
Don't worry, you'll do better next time


# ANALYZE RANDOM!
You can't analyze random bc it's random! But you can find "order" in "entropy" I guess. Like when you have your room messed up but somehow you can find everything. Nevermid.
Just look at the results below

In [72]:
dfRows.group_by(
  'reason'
).agg(
  pl.count('reason').alias('count')
).sort(
  "count", descending=True
)


reason,count
u8,u32
0,2271
1,134
4,22
2,16
3,6


In [73]:
dfRows.group_by(
  'shared_values_count'
).agg(pl.count('shared_values_count').alias('count'))

shared_values_count,count
u8,u32
3,31
0,1227
1,953
4,3
2,235


In [74]:
df_jumps_map = dfRows.group_by(
  'jumps_map'
).agg(pl.count('jumps_map').alias('count')).sort("count", descending=True)

display(df_jumps_map[0:10])

repeated_jumps_map = df_jumps_map.filter(pl.col("count") > 1)["jumps_map"].to_list()

q = dfRows.filter(
  pl.col("jumps_map").is_in(repeated_jumps_map) 
).sort("jumps_map")

display(q)

with open ("./jumps_map.json", "w") as file:
  json.dump(df_jumps_map["jumps_map"].to_list(), file)



jumps_map,count
str,u32
"""17_03_01_09_08""",2
"""01_04_05_23_05""",2
"""03_17_02_08_01""",2
"""05_06_01_08_03""",2
"""08_07_02_06_06""",2
"""07_05_11_04_01""",2
"""05_06_05_06_01""",1
"""06_03_04_04_17""",1
"""05_05_02_12_05""",1


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_left_b6,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,score_r8_b6,shared_values,shared_values_count,score_row,jumps_map
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,bool,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,str
"""09/11/2022""","""0932""","""06 16 39 11 07 44""","""43""","""26 35 12""","""Promoción Sí o Sí""","""060711163944""","[6, 7, … 44]",6,7,11,16,39,44,false,0,0.1667,0.025,0.025,0.027,0.0303,0.0909,0.1667,0.1111,0.0769,0.0286,0.0256,0.0909,[],0,0.8647,"""01_04_05_23_05"""
"""15/08/2018""","""0523""","""11 07 06 39 44 16""","""09""","""""","""""","""060711163944""","[6, 7, … 44]",6,7,11,16,39,44,false,0,0.1667,0.025,0.025,0.027,0.0303,0.0909,0.1667,0.1111,0.0769,0.0286,0.0256,0.0909,[],0,0.8647,"""01_04_05_23_05"""
"""15/01/2025""","""1160""","""36 16 47 38 19 46""","""32""","""18 24 43 08""","""Promoción Sí o Sí""","""161936384647""","[16, 19, … 47]",16,19,36,38,46,47,false,0,0.0556,0.0333,0.0357,0.0833,0.0909,0.25,0.0556,0.0294,0.0286,0.0238,0.0238,0.25,[],0,0.96,"""03_17_02_08_01"""
"""30/03/2014""","""0066""","""01 04 21 23 31 32""","""38""","""""","""""","""010421233132""","[1, 4, … 32]",1,4,21,23,31,32,false,0,0.3333,0.0222,0.0233,0.037,0.0385,0.0526,0.3333,0.0526,0.05,0.037,0.037,0.0526,[0.037],1,1.0694,"""03_17_02_08_01"""
"""17/05/2017""","""0393""","""34 23 22 16 11 31""","""37""","""29""","""Promoción Sí o Sí""","""111622233134""","[11, 16, … 34]",11,16,22,23,31,34,false,0,0.0667,0.0286,0.0323,0.0385,0.0385,0.0526,0.0667,0.05,0.05,0.037,0.0345,0.0526,[],0,0.548,"""05_06_01_08_03"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""23/11/2008""","""0973""","""01 08 13 24 28 29""","""45""","""12""","""Promoción Sí o Sí""","""010813242829""","[1, 8, … 29]",1,8,13,24,28,29,false,0,0.1429,0.0222,0.0256,0.0286,0.04,0.0455,0.1429,0.0909,0.0476,0.0417,0.0417,0.0455,[],0,0.7151,"""07_05_11_04_01"""
"""06/11/2022""","""0931""","""20 05 22 13 34 28""","""36""","""24""","""Promoción Sí o Sí""","""051320222834""","[5, 13, … 34]",5,13,20,22,28,34,false,0,0.0833,0.0244,0.0294,0.0357,0.037,0.0455,0.0833,0.0556,0.0526,0.0417,0.0345,0.0455,[],0,0.5685,"""08_07_02_06_06"""
"""27/01/2021""","""0746""","""22 34 20 13 28 05""","""24""","""""","""""","""051320222834""","[5, 13, … 34]",5,13,20,22,28,34,false,0,0.0833,0.0244,0.0294,0.0357,0.037,0.0455,0.0833,0.0556,0.0526,0.0417,0.0345,0.0455,[],0,0.5685,"""08_07_02_06_06"""


In [75]:
def show_b_stats (col: str):
  df_viz = dfRows.group_by(
    col
  ).agg(pl.count(col).alias('count'))

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)



In [76]:
show_b_stats('score_left_b1')
show_b_stats('score_r8_b1')


score_left_b1,count
f64,u32
0.0769,114
0.0588,93
0.125,142
0.0476,54
0.0333,8
…,…
0.0909,136
0.1667,115
0.0556,91


alt.Chart(...)

score_r8_b1,count
f64,u32
0.0333,8
0.0769,114
0.125,142
0.0588,93
0.0476,54
…,…
0.0909,136
0.0385,17
0.0556,91


alt.Chart(...)

In [77]:
show_b_stats('score_left_b2')
show_b_stats('score_r8_b2')

score_left_b2,count
f64,u32
0.025,160
0.0333,42
0.0233,234
0.0769,1
0.0588,1
…,…
0.0385,26
0.0244,209
0.0556,4


alt.Chart(...)

score_r8_b2,count
f64,u32
0.0588,126
0.125,81
0.0333,32
0.0769,94
0.0476,94
…,…
0.0909,111
0.0385,67
0.0526,115


alt.Chart(...)

In [78]:
show_b_stats('score_left_b3')
show_b_stats('score_r8_b3')

score_left_b3,count
f64,u32
0.0769,5
0.025,115
0.0588,8
0.0333,88
0.0476,31
…,…
0.0385,61
0.0227,71
0.0556,26


alt.Chart(...)

score_r8_b3,count
f64,u32
0.125,18
0.0588,96
0.0333,91
0.0769,57
0.025,5
…,…
0.0385,96
0.0244,1
0.0556,90


alt.Chart(...)

In [79]:
show_b_stats('score_left_b4')
show_b_stats('score_r8_b4')

score_left_b4,count
f64,u32
0.0769,26
0.125,2
0.0476,76
0.0588,58
0.025,56
…,…
0.0909,3
0.0227,6
0.0526,59


alt.Chart(...)

score_r8_b4,count
f64,u32
0.0769,17
0.0588,43
0.0476,78
0.025,46
0.125,3
…,…
0.0385,107
0.0244,8
0.0417,100


alt.Chart(...)

In [80]:
show_b_stats('score_left_b5')
show_b_stats('score_r8_b5')

score_left_b5,count
f64,u32
0.0476,121
0.025,15
0.125,18
0.0333,86
0.0233,2
…,…
0.0909,52
0.1667,5
0.0526,113


alt.Chart(...)

score_r8_b5,count
f64,u32
0.0233,16
0.0588,8
0.0476,26
0.0769,4
0.0333,102
…,…
0.0244,29
0.0227,6
0.0417,25


alt.Chart(...)

In [81]:
show_b_stats('score_left_b6')
show_b_stats('score_r8_b6')

score_left_b6,count
f64,u32
0.025,1
0.125,92
0.0769,141
0.0476,87
0.0333,34
…,…
0.0909,124
0.0385,63
0.0417,65


alt.Chart(...)

score_r8_b6,count
f64,u32
0.0588,103
0.125,92
0.0476,87
0.025,1
0.0333,34
…,…
0.1667,46
0.0909,124
0.0526,108


alt.Chart(...)